In [1]:
# Utiliser le drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Classification Random Forest

### Obtenir la classe des entrées

In [ ]:
import pandas as pd
import re
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

# Fonction pour obtenir la position : estive/lisière
def separate_integers_letters(string):
    integers = ''.join(re.findall(r'\d+', string))
    letters = ''.join(re.findall(r'[a-zA-Z]+', string))
    return integers, letters

os.chdir('/content/gdrive/MyDrive/cross_validation_spatial')

# Répertoire courant
current_directory = os.getcwd()

# Liste des dossiers
folders = [folder for folder in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, folder))]

folders = sorted(folders)

acccuracy_scores = []

mlb = MultiLabelBinarizer()

print(current_directory)
print(folders)

for folder_name in folders:

  # Chemin d'accès
  train_path = current_directory + "/" + folder_name + "/" + folder_name + "_train.csv"
  test_path =  current_directory + "/" + folder_name + "/" + folder_name + "_train.csv"

  df_train = pd.read_csv('/content/gdrive/MyDrive/cross_validation_spatial/Antras_Isard/Antras_Isard_train.csv')
  df_test = pd.read_csv('/content/gdrive/MyDrive/cross_validation_spatial/Antras_Isard/Antras_Isard_test.csv')

  # Détermination des quatre classes

  ## Train

  # Ajouter au dataframe la colonne dynamique et position
  dynamique = []
  position = []

  for index, row in df_train.iterrows():

      # La colonne où se trouve l'information
      station_name = row['station_name']
      name_split = station_name.split('/')

      # Localiser l'information d'intérêt
      info = name_split[2]
      info_split = info.split('_')

      # Déterminer la dynamique
      if info_split[2] == 'Bleu':
        dynamique.append('bleu')
      else:
        dynamique.append('rouge')

      # Déterminer la position
      integers, letters = separate_integers_letters(info_split[1])
      if letters == 'L':
        position.append('lisière')
      else:
        position.append('estive')

  # Ajouter la colonne dynamique et position
  df_train['dynamique'] = dynamique
  df_train['position'] = position

  # print(df_train)

  ## Test

  # Ajouter au dataframe la colonne dynamique et position
  dynamique = []
  position = []

  for index, row in df_test.iterrows():

      # La colonne où se trouve l'information
      station_name = row['station_name']
      name_split = station_name.split('/')

      # Localiser l'information d'intérêt
      info = name_split[2]
      info_split = info.split('_')

      # Déterminer la dynamique
      if info_split[2] == 'Bleu':
        dynamique.append('bleu')
      else:
        dynamique.append('rouge')

      # Déterminer la position
      integers, letters = separate_integers_letters(info_split[1])
      if letters == 'L':
        position.append('lisière')
      else:
        position.append('estive')

  # Ajouter la colonne dynamique et position
  df_test['dynamique'] = dynamique
  df_test['position'] = position

  # print(df_test)

  ## Train

  # ajout de la cible

  # one-hot encoded
  one_hot_encoded = pd.get_dummies(df_train[['dynamique', 'position']])

  # Concaténer le one-hot encoding
  df_train = pd.concat([df_train, one_hot_encoded], axis=1)

  # print(df_train)

  ## Test

  # ajout de la cible

  # one-hot encoded
  one_hot_encoded = pd.get_dummies(df_test[['dynamique', 'position']])

  # Concaténer le one-hot encoding
  df_test = pd.concat([df_test, one_hot_encoded], axis=1)

  # Enlever les colonnes inutiles

  df_train.drop(columns = ['file_name', 'station_name', 'dynamique', 'position'], inplace = True)

  df_test.drop(columns = ['file_name', 'station_name', 'dynamique', 'position'], inplace = True)

  X_train = df_train[['ZCR', 'MEANt',	'VARt',	'SKEWt','KURTt',	'LEQt',	'BGNt',	'SNRt',	'MED',	'ACTtFraction',	'ACTtCount',	'EVNtFraction',	'EVNtMean',	'EVNtCount',	'MEANf',	'VARf',	'SKEWf',	'KURTf',	'NBPEAKS',	'LEQf',	'ENRf',	'BGNf',	'SNRf',	'Hf',	'EAS',	'ECU',	'ECV',	'EPS',	'EPS_KURT',	'EPS_SKEW',	'ACI',	'NDSI',	'rBA',	'AnthroEnergy',	'BioEnergy',	'BI',	'ROU',	'ADI',	'AEI',	'LFC',	'MFC',	'HFC',	'ACTspFract',	'ACTspCount',	'ACTspMean',	'EVNspFract',	'EVNspMean',	'EVNspCount',	'H_Havrda',	'H_Renyi',	'H_pairedShannon',	'H_gamma',	'H_GiniSimpson',	'RAOQ',	'AGI',	'ROItotal',	'ROIcover']]
  Y_train = df_train[['dynamique_bleu',	'dynamique_rouge',	'position_estive',	'position_lisière']]

  # X_train, _, Y_train, _ = train_test_split(X_train, Y_train, test_size=0)


  # model = RandomForestClassifier()
  # model = GradientBoostingClassifier()
  model = MLPClassifier(hidden_layer_sizes=(64, 128), max_iter=500)
  model.fit(X_train, Y_train)

  # print(df_test)

  X_test = df_test[['ZCR', 'MEANt',	'VARt',	'SKEWt','KURTt',	'LEQt',	'BGNt',	'SNRt',	'MED',	'ACTtFraction',	'ACTtCount',	'EVNtFraction',	'EVNtMean',	'EVNtCount',	'MEANf',	'VARf',	'SKEWf',	'KURTf',	'NBPEAKS',	'LEQf',	'ENRf',	'BGNf',	'SNRf',	'Hf',	'EAS',	'ECU',	'ECV',	'EPS',	'EPS_KURT',	'EPS_SKEW',	'ACI',	'NDSI',	'rBA',	'AnthroEnergy',	'BioEnergy',	'BI',	'ROU',	'ADI',	'AEI',	'LFC',	'MFC',	'HFC',	'ACTspFract',	'ACTspCount',	'ACTspMean',	'EVNspFract',	'EVNspMean',	'EVNspCount',	'H_Havrda',	'H_Renyi',	'H_pairedShannon',	'H_gamma',	'H_GiniSimpson',	'RAOQ',	'AGI',	'ROItotal',	'ROIcover']]
  Y_test = df_test[['dynamique_bleu',	'dynamique_rouge',	'position_estive',	'position_lisière']]

  print(Y_test)

  # _, X_test, _, Y_test = train_test_split(X_test, Y_test, test_size=1)

  # Prédictions sur l'ensemble de test
  Y_pred = model.predict(X_test)

  # Evaluer les performances du modèle
  accuracy = accuracy_score(Y_test, Y_pred)
  print("Accuracy de la station " + folder_name + " :", accuracy)

  acccuracy_scores.append(accuracy)

print(acccuracy_scores)

# # Premier training sur la station Antras_Isard
# df_train = pd.read_csv('/content/gdrive/MyDrive/cross_validation_spatial/Antras_Isard/Antras_Isard_train.csv')
# df_test = pd.read_csv('/content/gdrive/MyDrive/cross_validation_spatial/Antras_Isard/Antras_Isard_test.csv')

# # Détermination des quatre classes

# ## Train

# # Ajouter au dataframe la colonne dynamique et position
# dynamique = []
# position = []

# for index, row in df_train.iterrows():

#     # La colonne où se trouve l'information
#     station_name = row['station_name']
#     name_split = station_name.split('/')

#     # Localiser l'information d'intérêt
#     info = name_split[2]
#     info_split = info.split('_')

#     # Déterminer la dynamique
#     if info_split[2] == 'Bleu':
#       dynamique.append('bleu')
#     else:
#       dynamique.append('rouge')

#     # Déterminer la position
#     integers, letters = separate_integers_letters(info_split[1])
#     if letters == 'L':
#       position.append('lisière')
#     else:
#       position.append('estive')

# # Ajouter la colonne dynamique et position
# df_train['dynamique'] = dynamique
# df_train['position'] = position

# print(df_train)

# ## Test

# # Ajouter au dataframe la colonne dynamique et position
# dynamique = []
# position = []

# for index, row in df_test.iterrows():

#     # La colonne où se trouve l'information
#     station_name = row['station_name']
#     name_split = station_name.split('/')

#     # Localiser l'information d'intérêt
#     info = name_split[2]
#     info_split = info.split('_')

#     # Déterminer la dynamique
#     if info_split[2] == 'Bleu':
#       dynamique.append('bleu')
#     else:
#       dynamique.append('rouge')

#     # Déterminer la position
#     integers, letters = separate_integers_letters(info_split[1])
#     if letters == 'L':
#       position.append('lisière')
#     else:
#       position.append('estive')

# # Ajouter la colonne dynamique et position
# df_test['dynamique'] = dynamique
# df_test['position'] = position

# print(df_test)

/content/gdrive/MyDrive/cross_validation_spatial
['Antras_Isard', 'Appy_Etang_dAppy', 'Arrien_Pic_Midi_Bordes', 'Ascou_Pailheres', 'Aston_Plateau_de_Beille', 'Aulus_Turon_Rose', 'Bestiac_Trimouns', 'Bethmale_Ayet', 'Bethmale_Eychelle', 'Bonac_Arech', 'Bonac_Orle', 'Couflens_Pouill', 'Couflens_Saubé', 'Formiguères_Serra_dels_Alarbs', 'Gestiès_Col_du_Sasc', 'Gestiès_Le_Clot', 'Goulier_Val_de_Sos', 'Lercoul_Col_de_Grail', 'Les_Angles_Puig_del_Pam', 'Les_Angles_Roc_dAude', 'Merens_Comte', 'Miglos_Pla_de_Montcamp', 'Mijanes_Estagnet', 'Mijanes_Trabesses', 'Montferrier_Mont_dOlmes', 'Montsegur_Frau', 'Montségur_Soularac', 'Orgeix_Coume', 'Orlu_RNCFS', 'Saint-Lary_Herbe_Soulette', 'Seix_Col_de_Pause', 'Seix_Pic_de_Montbuou', 'Sentein_Eylie', 'Ustou_Col_dEscots', 'Ustou_Guzet-Neige', 'Valcebollère_Pla_des_Salines']
      dynamique_bleu  dynamique_rouge  position_estive  position_lisière
0                  1                0                0                 1
1                  1         

## One-hot encoding

In [ ]:
## Train

# ajout de la cible

# one-hot encoded
one_hot_encoded = pd.get_dummies(df_train[['dynamique', 'position']])

# Concaténer le one-hot encoding
df_train = pd.concat([df_train, one_hot_encoded], axis=1)

print(df_train)

## Test

# ajout de la cible

# one-hot encoded
one_hot_encoded = pd.get_dummies(df_test[['dynamique', 'position']])

# Concaténer le one-hot encoding
df_test = pd.concat([df_test, one_hot_encoded], axis=1)

print(df_test)

                          file_name  \
0      SMA05744_20220529_160102.wav   
1      SMA05744_20220529_163102.wav   
2      SMA05744_20220529_170102.wav   
3      SMA05744_20220529_173102.wav   
4      SMA05744_20220529_180102.wav   
...                             ...   
29157  SMA08828_20230708_070000.wav   
29158  SMA08828_20230708_080000.wav   
29159  SMA08828_20230708_090000.wav   
29160  SMA08828_20230708_100000.wav   
29161  SMA08828_20230708_110000.wav   

                                            station_name          ZCR  \
0            2022/Ascou_Pailheres/SM5744_529E_Bleu/Data/  1764.416667   
1            2022/Ascou_Pailheres/SM5744_529E_Bleu/Data/  1816.266667   
2            2022/Ascou_Pailheres/SM5744_529E_Bleu/Data/  2054.350000   
3            2022/Ascou_Pailheres/SM5744_529E_Bleu/Data/  1430.466667   
4            2022/Ascou_Pailheres/SM5744_529E_Bleu/Data/  1308.400000   
...                                                  ...          ...   
29157  2023/Valcebol

## Drop it like it's hot

In [ ]:
# Enlever les colonnes inutiles

df_train.drop(columns = ['file_name', 'station_name', 'dynamique', 'position'], inplace = True)

df_test.drop(columns = ['file_name', 'station_name', 'dynamique', 'position'], inplace = True)

df_train

df_test

,ZCR,MEANt,VARt,SKEWt,KURTt,LEQt,BGNt,SNRt,MED,Ht,...,H_gamma,H_GiniSimpson,RAOQ,AGI,ROItotal,ROIcover,dynamique_bleu,dynamique_rouge,position_estive,position_lisière
0,26.500000,-0.071138,0.000943,18.844282,741.878546,70.804882,-20.364283,3.631517,0.095893,0.768039,...,42662.896368,0.969171,0.652523,3.908220,20,4.562954,1,0,0,1
1,0.000000,-0.072088,0.000051,0.003911,3.185880,70.220556,-20.692665,0.043466,0.092335,0.999543,...,19461.220470,0.988569,0.890561,1.281302,71,4.644769,1,0,0,1
2,1.700000,-0.072561,0.000240,-0.026333,4.415867,70.428267,-18.839075,0.173548,0.114300,0.995125,...,10507.448100,0.972643,0.408822,1.359625,78,1.551557,1,0,0,1
3,0.166667,-0.072394,0.000162,-0.011491,4.044512,70.347133,-19.333529,0.093239,0.107975,0.997016,...,12718.746077,0.981275,0.570758,1.323778,243,2.307376,1,0,0,1
4,300.900000,-0.072697,0.000586,-0.046556,6.230253,70.707393,-17.271708,0.602948,0.136904,0.985898,...,6822.684806,0.972457,0.225787,1.339269,87,0.479735,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,2696.983333,0.004096,0.000041,-0.013572,3.080608,50.621266,-33.148483,0.215997,0.022008,0.994540,...,21687.891847,0.987560,0.540549,1.374001,261,7.617604,1,0,1,0
1383,1294.616667,0.003861,0.000054,-0.019291,3.605126,51.407518,-33.086456,0.498210,0.022165,0.989018,...,11448.065278,0.973401,0.133567,1.287860,270,4.937008,1,0,1,0
1384,1902.233333,0.005184,0.000042,-0.006777,3.162574,51.394935,-32.825462,0.259311,0.022842,0.994072,...,17757.021786,0.984376,0.434834,1.313709,351,5.550669,1,0,1,0
1385,4599.466667,0.004972,0.000078,-0.004408,3.483235,53.125420,-30.525963,0.349786,0.029765,0.990836,...,19107.422964,0.988482,0.873058,1.319700,293,3.422659,1,0,1,0


## Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = df_train[['ZCR', 'MEANt',	'VARt',	'SKEWt','KURTt',	'LEQt',	'BGNt',	'SNRt',	'MED',	'ACTtFraction',	'ACTtCount',	'EVNtFraction',	'EVNtMean',	'EVNtCount',	'MEANf',	'VARf',	'SKEWf',	'KURTf',	'NBPEAKS',	'LEQf',	'ENRf',	'BGNf',	'SNRf',	'Hf',	'EAS',	'ECU',	'ECV',	'EPS',	'EPS_KURT',	'EPS_SKEW',	'ACI',	'NDSI',	'rBA',	'AnthroEnergy',	'BioEnergy',	'BI',	'ROU',	'ADI',	'AEI',	'LFC',	'MFC',	'HFC',	'ACTspFract',	'ACTspCount',	'ACTspMean',	'EVNspFract',	'EVNspMean',	'EVNspCount',	'H_Havrda',	'H_Renyi',	'H_pairedShannon',	'H_gamma',	'H_GiniSimpson',	'RAOQ',	'AGI',	'ROItotal',	'ROIcover']]
Y_train = df_train[['dynamique_bleu',	'dynamique_rouge',	'position_estive',	'position_lisière']]


model = RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier()

## Test

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

X_test = df_test[['ZCR', 'MEANt',	'VARt',	'SKEWt','KURTt',	'LEQt',	'BGNt',	'SNRt',	'MED',	'ACTtFraction',	'ACTtCount',	'EVNtFraction',	'EVNtMean',	'EVNtCount',	'MEANf',	'VARf',	'SKEWf',	'KURTf',	'NBPEAKS',	'LEQf',	'ENRf',	'BGNf',	'SNRf',	'Hf',	'EAS',	'ECU',	'ECV',	'EPS',	'EPS_KURT',	'EPS_SKEW',	'ACI',	'NDSI',	'rBA',	'AnthroEnergy',	'BioEnergy',	'BI',	'ROU',	'ADI',	'AEI',	'LFC',	'MFC',	'HFC',	'ACTspFract',	'ACTspCount',	'ACTspMean',	'EVNspFract',	'EVNspMean',	'EVNspCount',	'H_Havrda',	'H_Renyi',	'H_pairedShannon',	'H_gamma',	'H_GiniSimpson',	'RAOQ',	'AGI',	'ROItotal',	'ROIcover']]
Y_test = df_test[['dynamique_bleu',	'dynamique_rouge',	'position_estive',	'position_lisière']]

# Prédictions sur l'ensemble de test
Y_pred = model.predict(X_test)

# Evaluer les performances du modèle
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(Y_test, Y_pred))



Accuracy: 0.26820475847152125
              precision    recall  f1-score   support

           0       0.49      0.58      0.53       796
           1       0.24      0.17      0.20       591
           2       0.70      0.46      0.55       758
           3       0.55      0.76      0.64       629

   micro avg       0.51      0.50      0.50      2774
   macro avg       0.49      0.49      0.48      2774
weighted avg       0.51      0.50      0.49      2774
 samples avg       0.51      0.50      0.50      2774



AttributeError: 'DataFrame' object has no attribute 'argmax'